In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NorthCorp")
import datetime
import matplotlib.pyplot as plt
import os

In [2]:
base_dir = r"/home/parth/Machine Learning/Datasets/North Corp/Ajanta"
files = os.listdir(base_dir)
files_corrected = []
data = []
first_one = True
for file in files :
    if file.endswith(".csv") :
        files_corrected.append(file)
        dat = pd.read_csv(os.path.join( "/home/parth/Machine Learning/Datasets/North Corp/Ajanta" , file))
        if first_one :
            columns = dat.columns
        else :
            assert  np.all(columns == dat.columns)
        first_one = False
        data.append(dat)
        
files = files_corrected
del files_corrected

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
for i in range(len(data)) :
    data[i] = data[i].sort_values("updateddate" , ascending=True)

for i in range(len(data)) :
    data[i]["dateSeconds"] = data[i]["updateddate"].apply(
        lambda x: datetime.datetime.strptime(x , "%Y-%m-%d %H:%M:%S")).apply(
        lambda x: (x - datetime.datetime(1970,1,1)).total_seconds() )

In [4]:
for i in range(len(data)) :
    previdletime = data[i].iloc[0]["time_idle"]
    data[i]["idleWastedFuel"] = None
    data[i]["idleTimeDelta"] = None
    for row in range(1 , len(data[i])):
        currentidletime = data[i].iloc[row]["time_idle"]
        currentFuel = data[i].iloc[row]["total_fuel_consumption"]
        if (currentidletime != previdletime):
            if row != 0 :
                data[i].at[row , "idleWastedFuel"] = abs ((currentFuel - prevFuel))
                data[i].at[row  , "idleTimeDelta"] = abs(currentidletime - previdletime)
            else :
                data[i].at[row , "idleWastedFuel"] = 0
        else:
            data[i].at[row , "idleWastedFuel"] = 0
        previdletime = currentidletime
        prevFuel = currentFuel
    data[i].at[0 , "idleWastedFuel"] = 0
    data[i].at[0 , "idleTimeDelta"] = 0

In [5]:
IdleCases = []

for i in range(len(data)) :
    IdleCases.append(data[i][data[i].speed == 0])
    IdleCases[i] = IdleCases[i][IdleCases[i]["idleWastedFuel"] != 0]

In [6]:
StopCases = []

for i in range(len(data)) :
    StopCases.append(data[i][data[i].speed == 0])
    StopCases[i] = StopCases[i][StopCases[i]["idleWastedFuel"] == 0 ]
    StopCases[i]["stopTimeDelta"] = 0
    StopCases[i].drop(columns=["idleWastedFuel" , "idleTimeDelta"] , inplace=True)
    StopCases[i].stopTimeDelta = StopCases[i].dateSeconds.diff()
    StopCases[i].at[0 , "stopTimeDelta"] = 0
    StopCases[i]["stopTimeDelta"] = StopCases[i]["stopTimeDelta"] / 3600.0

In [7]:
for i in range(len(data)) :
    print("Total Fuel Wasted CASE:{} : {} ".format(i , sum(IdleCases[i]["idleWastedFuel"] )))

Total Fuel Wasted CASE:0 : 1639.4899999999507 
Total Fuel Wasted CASE:1 : 418.5200000000259 
Total Fuel Wasted CASE:2 : 437.6900000000205 
Total Fuel Wasted CASE:3 : 3744.7300000000578 
Total Fuel Wasted CASE:4 : 422.1100000000697 
Total Fuel Wasted CASE:5 : 8701.790000000015 


In [8]:
for i in range(len(data)) :
    IdleCases[i].to_csv("IdleCases-{}.csv".format(files[i]))
    StopCases[i].to_csv("StopCases-{}.csv".format(files[i]))
    data[i].to_csv("Full-Data-{}.csv".format(files[i]))

In [9]:
data[0].columns


Index(['Unnamed: 0', 'positionid', 'vehicleid', 'gtpdriverid', 'version',
       'heading', 'latitude', 'longitude', 'altitude', 'address',
       'updateddate', 'total_fuel_consumption', 'total_engine_hours',
       'total_distance', 'time_sweetspot', 'time_idle', 'time_pto',
       'time_drive', 'speed', 'msg_trigger_type', 'current_fuel_level',
       'received_date', 'dateSeconds', 'idleWastedFuel', 'idleTimeDelta'],
      dtype='object')

In [10]:
data[0]["time_sweetspot"]

0        367.30
1        367.30
2        367.30
3        367.30
4        367.30
5        367.30
6        367.30
7        367.30
8        367.30
9        367.30
10       367.30
11       367.30
12       367.30
13       367.30
14       367.30
15       367.30
16       367.30
17       367.30
18       367.30
19       367.30
20       367.30
21       367.30
22       367.30
23       367.30
24       367.30
25       367.30
26       367.30
27       367.30
28       367.30
29       367.30
          ...  
42379    774.55
42380    774.55
42381    774.55
42382    774.55
42383    774.55
42384    774.55
42385    774.55
42386    774.55
42387    774.55
42388    774.55
42389    774.55
42390    774.55
42391    774.55
42392    774.55
42393    774.55
42394    774.55
42395    774.55
42396    774.55
42397    774.55
42398    774.55
42399    774.55
42400    774.55
42401    774.55
42402    774.55
42403    774.55
42404    774.55
42405    774.55
42406    774.55
42407    774.55
42408    774.55
Name: time_sweetspot, Le